### Notebook使用的通用说明
ModelScope 平台也提供在线的运行环境，您可直接在Notebook中运行，Notebook中提供官方镜像无需自主进行环境安装，更加方便快捷，推荐大家使用！
注意：该功能需要您登录后使用，**新用户注册ModelScope账号后即可获得免费算力资源**，详情请参阅[免费额度说明 ](../Notebook/免费额度说明.ipynb)。

![image.png](./resources/1659688356174-2bcf4652-0221-42fa-8985-10c063c4dac5.png)

成功登录后，您可直接在模型详情页中，点击“在Notebook中打开”，并选择需要运行的实例环境（CPU环境或GPU环境）

![image.png](./resources/1659696583113-88d50b88-a0a0-41e0-afa1-77e06b472c8a.png)
![image.png](./resources/1659688205573-63bc9a9f-162d-4c5c-ae02-64c5b2699ca3.png)

系统将预装ModelScope官方镜像，点击“启动”按钮即可在Notebook中使用。此时实例启动需要一些时间，请耐心等待，当环境启动后点击“查看Notebook”，则跳转notebook页面，即可在新环境中使用。

![image.png](./resources/1659690224978-3b2db930-692c-48ec-9035-784be2b4dbe5.png)

系统将新开一个页面，点击Python3，进入为您搭建好的Python3开发环境。

![image.png](./resources/1659690265185-3d141b84-9207-4b4d-8c25-6a20a2c854af.png)

复制模型详情页面里快速使用的代码到该python环节中，点击运行按钮，即可跑出推理结果。（一些模型的使用方法，可参看具体的模型卡片的README信息中的代码要求）

![image.png](./resources/1659696610207-71cae764-80b2-4a8f-a6d9-d0abc6a61c88.png)
![image.png](./resources/1659690976895-4ac3c563-81a2-4b09-a433-b54b328ebc7d.png)

您也可使用该环节实现您的代码编程。关于Notebook功能，更多详情参阅以下文档：

- [Notebook 功能概述 ](../Notebook/Notebook%20功能概述.ipynb)
- [创建及管理 Notebook 实例 ](../Notebook/Notebook用户操作指南/创建及管理%20Notebook%20实例.ipynb)
- [读写数据与文件传输 ](../Notebook/Notebook用户操作指南/读写数据与文件传输.ipynb)
- [管理第三方库 ](../Notebook/Notebook用户操作指南/管理第三方库.ipynb)
- [免费资源用尽与数据迁移](../Notebook/Notebook用户操作指南/免费资源额度用尽和数据迁移.ipynb)
### 在Notebook运行一个example
通过ModelScope Notebook使用ModelScope时，所有的依赖环境都已经预先安装好，可以直接使用。
#### 运行推理pipeline
下面以中文分词任务为例，说明pipeline函数的基本用法。

1.  pipeline函数支持指定特定任务名称，加载任务默认模型，创建对应pipeline对象。
执行如下python代码 ：


In [1]:
from modelscope.pipelines import pipeline
word_segmentation = pipeline('word-segmentation')


 

2.  输入文本 


In [1]:
input_str = '今天天气不错，适合出去游玩'
print(word_segmentation(input_str))
{'output': '今天 天气 不错 ， 适合 出去 游玩'}


 

3.  输入多条样本 

pipeline对象也支持传入多个样本列表输入，返回对应输出列表，每个元素对应输入样本的返回结果。



In [1]:
inputs =  ['今天天气不错，适合出去游玩','这本书很好，建议你看看']
print(word_segmentation(inputs))
[{'output': '今天 天气 不错 ， 适合 出去 游玩'}, {'output': '这 本 书 很 好 ， 建议 你 看看'}]



#### 运行finetune训练和评估
下面以一个简单的文本分类任务为例，演示如何通过十几行代码，就可以端到端拉起一个finetune任务。整体流程包含一下步骤：

- 载入数据集
- 数据预处理
- 训练
- 评估
#### 载入数据集
ModelScope可以提供了标准的`MsDataset`接口供用户进行基于ModelScope生态的数据源加载。下面以加载NLP领域的afqmc（Ant Financial Question Matching Corpus）数据集为例进行演示


In [1]:
from modelscope.msdatasets import MsDataset
# 载入训练数据
train_dataset = MsDataset.load('afqmc_small', split='train')
# 载入评估数据
eval_dataset = MsDataset.load('afqmc_small', split='validation')



#### 数据预处理
在ModelScope中，数据预处理与模型强相关，因此，在指定模型以后，ModelScope框架会自动从对应的modelcard中读取配置文件中的preprocessor关键字，自动完成预处理的实例化。


In [1]:
# 指定文本分类模型
model_id = 'damo/nlp_structbert_sentence-similarity_chinese-base'


#### 训练
首先，配置训练所需参数：


In [1]:
from modelscope.trainers import build_trainer

# 指定工作目录
tmp_dir = "/tmp"

# 配置参数
kwargs = dict(
        model=model_id,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        work_dir=tmp_dir)



其次，根据参数实例化trainer对象


In [1]:
trainer = build_trainer(default_args=kwargs)



最后，调用train接口进行训练


In [1]:
trainer.train()



恭喜，你完成了一次模型训练😀
## 评估
训练完成以后，配置评估数据集，直接调用trainer对象的evaluate函数，即可完成模型的评估，


In [1]:
# 直接调用trainer.evaluate，可以传入train阶段生成的ckpt
# 也可以不传入参数，直接验证model
metrics = trainer.evaluate(checkpoint_path=None)
print(metrics)
